In [31]:
# Include necessary imports
import os
import torch 
import pandas as pd
from torch.utils.data import DataLoader
from music21 import *
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler



In [15]:
# Preprocess the data

folder_path = 'Data/'
test = []
train = []
validation = []
for dirname in os.listdir(folder_path):
    if dirname != '.DS_Store':
        for filename in os.listdir(folder_path + dirname):
            df = pd.read_csv(folder_path + dirname + '/' + filename)
            transposed_df = df.transpose()
            if dirname == 'test':
                test.append(transposed_df)
            if dirname == 'train':
                train.append(transposed_df)
            if dirname == 'valid':
                validation.append(transposed_df)

# Model

In [16]:
class Model(torch.nn.Module):
    def __init__(self, input_size, output_size, hidden_dim=50, n_layers=2):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.lstm = torch.nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, output_size)
        
    def forward(self, x, hidden=None):
        lstm_output, (h,c) = self.lstm(x, hidden)
        model_output = self.fc(lstm_output)
        return model_output


# Train

In [20]:
def train_model(model, melody, harmonies, optimizer, criterion, num_epochs):
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        output = model(melody)
        loss = criterion(output, harmonies)
        loss.backward()
        optimizer.step()
        if (epoch + 1) % 100 == 0:
            print("Epoch: ", epoch, "Loss: ", loss)

In [54]:
criterion = torch.nn.MSELoss()
epoch = 5000
index = 0
#for song in train:
#index = index + 1
#print("training song ", index)
song = train[0]
scaler_melody = MinMaxScaler()
scaler_harmonies = MinMaxScaler()
melody = torch.tensor(scaler_melody.fit_transform(song.iloc[0].values.reshape(-1,1)), dtype=torch.float32).unsqueeze(0).reshape(1,song.shape[1],1)
harmonies = torch.tensor(scaler_harmonies.fit_transform(song.iloc[1:].values.T), dtype=torch.float32).unsqueeze(0)
model = Model(1, harmonies.shape[2])
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
train_model(model, melody, harmonies, optimizer, criterion, epoch)

Epoch:  99 Loss:  tensor(0.0573, grad_fn=<MseLossBackward0>)
Epoch:  199 Loss:  tensor(0.0422, grad_fn=<MseLossBackward0>)
Epoch:  299 Loss:  tensor(0.0225, grad_fn=<MseLossBackward0>)
Epoch:  399 Loss:  tensor(0.0145, grad_fn=<MseLossBackward0>)
Epoch:  499 Loss:  tensor(0.0079, grad_fn=<MseLossBackward0>)
Epoch:  599 Loss:  tensor(0.0137, grad_fn=<MseLossBackward0>)
Epoch:  699 Loss:  tensor(0.0050, grad_fn=<MseLossBackward0>)
Epoch:  799 Loss:  tensor(0.0032, grad_fn=<MseLossBackward0>)
Epoch:  899 Loss:  tensor(0.0022, grad_fn=<MseLossBackward0>)
Epoch:  999 Loss:  tensor(0.0458, grad_fn=<MseLossBackward0>)
Epoch:  1099 Loss:  tensor(0.0233, grad_fn=<MseLossBackward0>)
Epoch:  1199 Loss:  tensor(0.0067, grad_fn=<MseLossBackward0>)
Epoch:  1299 Loss:  tensor(0.0029, grad_fn=<MseLossBackward0>)
Epoch:  1399 Loss:  tensor(0.0021, grad_fn=<MseLossBackward0>)
Epoch:  1499 Loss:  tensor(0.0046, grad_fn=<MseLossBackward0>)
Epoch:  1599 Loss:  tensor(0.0016, grad_fn=<MseLossBackward0>)
Epo

# Hyperparameter 

In [ ]:
learning = [0.01,0.001]
n_layers= [1,2,3,4,5]
epochs= [100,1000,5000]
best_loss = float('inf')
best_params = {}


for LR in learning:
    for n_layer in n_layers:
        for epoch in epochs:
            print(f"Training with LR={LR} and n_layers={n_layer} and epochs={epoch}")
            model = Model(input_size=1, output_size=harmonies.shape[2], n_layers=n_layer)
            optimizer = torch.optim.Adam(model.parameters(), lr=LR)
            train_model(model, melody, harmonies, optimizer, criterion, epoch)
            with torch.no_grad():
                output = model(melody)
                loss = criterion(output, harmonies)
                print(f"Final Loss: {loss.item()}")        
            # Keep track of the best model (with lowest loss)
            if loss.item() < best_loss:
                best_loss = loss.item()
                best_params = {'learning': LR, 'n_layers': n_layer, 'epochs': epoch}
print("BEST: ", best_params)

In [ ]:
# tiny = pd.DataFrame([[67,62,59,43], [68,62,59,43]]).transpose()
# melody = torch.tensor(tiny.iloc[0], dtype=torch.float32).unsqueeze(0).reshape(1,2,1)
# harmonies = torch.transpose(torch.tensor(tiny.iloc[1:].values, dtype=torch.float32),0,1).unsqueeze(0)
# model = Model(1, harmonies.shape[2])
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# criterion = torch.nn.MSELoss()
# train_model(model, melody, harmonies, optimizer, criterion, 10000)

In [56]:
melody = train[1].iloc[0]
result = model(torch.tensor(melody, dtype=torch.float32).unsqueeze(0).reshape(1, train[1].shape[1], 1))
result_numpy = result.detach().numpy()
inversed = scaler_harmonies.inverse_transform(np.squeeze(result_numpy)).T
print(inversed)

score = stream.Score()
melody_part = stream.Part()
alto_part = stream.Part()
tenor_part = stream.Part()
bass_part = stream.Part()

for pitch in melody:
    melody_note = note.Note(int(pitch))
    melody_part.append(melody_note)

alto_notes = inversed[0]
tenor_notes = inversed[1]
bass_notes = inversed[2]  

for pitch in alto_notes:
    alto_note = note.Note(int(pitch.item()))
    alto_part.append(alto_note)
for pitch in tenor_notes:
     tenor_note = note.Note(int(pitch.item()))
     tenor_part.append(tenor_note)
for pitch in bass_notes:
    bass_note = note.Note(int(pitch.item()))
    bass_part.append(bass_note)

score.append(melody_part)
score.append(alto_part)
score.append(tenor_part)
score.append(bass_part)
score.show('midi')
score.write('musicxml', 'output.xml')

[[69.86145  71.8787   72.12315  72.13968  72.12614  72.12902  72.1289
  72.128944 72.128944 72.128944 72.128944 72.128944 72.14373  72.14087
  72.14098  72.140945 72.16598  72.161804 72.161995 72.16194  72.18089
  72.178185 72.17832  72.178276 72.15929  72.16204  72.144745 72.14761
  72.13953  72.14099  72.14093  72.14095  72.12612  72.12903  72.12892
  72.12896  72.14374  72.140884 72.14099  72.14096  72.14096  72.14096
  72.14096  72.14096  72.14096  72.14096  72.14096  72.14096  72.11255
  72.11847  72.11192  72.11351  72.10158  72.10464  72.10453  72.104576
  72.09363  72.096695 72.0966   72.09665  72.107544 72.104515 72.10462
  72.104576 72.12269  72.11824  72.11839  72.11833  72.14655  72.140816
  72.141045 72.14097  72.166016 72.16184  72.14487  72.14763  72.13954
  72.141014 72.14095  72.14097  72.12614  72.12904  72.12894  72.12897
  72.14376  72.1409   72.141014 72.140976 72.140976 72.140976 72.140976
  72.140976 72.11256  72.118484 72.118256 72.11835  72.11835  72.11835
  72

WindowsPath('C:/Users/foodrunner/CS370/PolyphAI/Code/output.xml')

In [ ]:
# Finetune (hyperparameters, move around test data (refer to notes), etc)

In [ ]:
# Test with new data + evaluate

In [ ]:
# Make any other changes

In [ ]:
# Sheet music + audio (musicAI)

In [ ]:
# Create new models if time permits (follow steps 3 - 7)

In [ ]:
# Compare models

In [ ]:
# Front end ** if time permits
# - Interactive sheet music
# - musescore front end??